In [6]:
# -*- coding: utf-8 -*-
r"""
hfl/chinese-bert-wwm-ext 微信聊天意图识别基线训练代码（Notebook 专用版）
数据路径：
C:\Users\ASUS\Desktop\Yuzuriha-Rin\assets\models\few_shot_intent_sft\data\
"""

import json
from collections import Counter
from pathlib import Path
import pandas as pd
import numpy as np
from datasets import Dataset, DatasetDict
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    set_seed,
)
from sklearn.metrics import accuracy_score, f1_score

# =================================== 配置 ===================================

# 数据路径（你的真实路径）
DATA_DIR = Path(
    r"C:\Users\ASUS\Desktop\Yuzuriha-Rin\assets\models\few_shot_intent_sft\data"
)
TELEMARKETING_DATA = DATA_DIR / "telemarketing_intent_cn.jsonl"
CROSSWOZ_DATA = DATA_DIR / "crosswoz.jsonl"

# 输出路径
OUTPUT_DIR = Path(
    r"C:\Users\ASUS\Desktop\Yuzuriha-Rin\assets\models\chinese_bert_baseline"
)
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# 训练超参数
MODEL_NAME = "hfl/chinese-bert-wwm-ext"
MAX_LENGTH = 128
BATCH_SIZE = 32
EPOCHS = 4
LEARNING_RATE = 2e-5
MIN_SAMPLES = 20
MAX_SAMPLES_PER_INTENT = 300
USE_CROSSWOZ = True
RANDOM_SEED = 42

set_seed(RANDOM_SEED)

# =================================== 黑名单 ===================================

BLACKLIST_INTENTS = {
    "查询类",
    "查询(产品信息)",
    "查询(价格)",
    "查询(优惠)",
    "查询(库存)",
    "查询(物流)",
    "查询(订单)",
    "查询(账户)",
    "查询(余额)",
    "实体(产品)",
    "实体(价格)",
    "实体(时间)",
    "实体(地点)",
    "实体(人名)",
    "实体(公司)",
    "实体识别",
    "产品推荐",
    "促销活动",
    "优惠信息",
    "下单",
    "支付",
    "退款",
    "投诉",
    "售后",
    "政治敏感",
    "污言秽语",
    "色情低俗",
    "暴力血腥",
    "违法犯罪",
    "广告营销",
    "诈骗信息",
    "肯定(没问题)",
    "否定(没有)",
    "转人工",
    "挂断电话",
    "保持通话",
    "重复",
    "澄清",
    "确认信息",
    "核实身份",
    "录音提示",
    "系统提示",
}

# =================================== 数据加载 ===================================


def load_jsonl(file_path):
    if not file_path.exists():
        print(f"⚠️ 文件不存在（跳过）: {file_path}")
        return pd.DataFrame()
    print(f"📂 正在加载: {file_path.name}")
    data = []
    with open(file_path, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if line:
                item = json.loads(line)
                data.append({"text": item["text"].strip(), "label": item["label"]})
    return pd.DataFrame(data)


def prepare_dataset():
    print("\n📊 开始准备数据集...")

    df_main = load_jsonl(TELEMARKETING_DATA)

    if USE_CROSSWOZ and CROSSWOZ_DATA.exists():
        df_extra = load_jsonl(CROSSWOZ_DATA)
        if not df_extra.empty:
            # CrossWOZ 意图映射
            mapping = {"greet": "招呼用语", "thank": "礼貌用语", "bye": "结束用语"}
            df_extra["label"] = df_extra["label"].map(mapping)
            df_extra = df_extra.dropna(subset=["label"])
            df = pd.concat([df_main, df_extra], ignore_index=True)
            print(f"✅ 已合并 CrossWOZ，总样本: {len(df)}")
        else:
            df = df_main
    else:
        df = df_main

    # 黑名单过滤
    before = len(df)
    df = df[~df["label"].isin(BLACKLIST_INTENTS)]
    print(f"   黑名单过滤: {before} → {len(df)}")

    # 样本阈值过滤
    intent_counts = Counter(df["label"])
    valid_intents = {k for k, v in intent_counts.items() if v >= MIN_SAMPLES}
    df = df[df["label"].isin(valid_intents)]
    print(f"   样本阈值过滤后: {len(df)} 样本，{len(valid_intents)} 意图")

    # 数据平衡
    balanced = []
    for intent in df["label"].unique():
        group = df[df["label"] == intent]
        if len(group) > MAX_SAMPLES_PER_INTENT:
            group = group.sample(n=MAX_SAMPLES_PER_INTENT, random_state=RANDOM_SEED)
        balanced.append(group)
    df = pd.concat(balanced, ignore_index=True)
    print(f"   数据平衡后: {len(df)} 样本")

    # 标签映射
    intents = sorted(df["label"].unique())
    label2id = {label: idx for idx, label in enumerate(intents)}
    id2label = {idx: label for label, idx in label2id.items()}

    df["label_id"] = df["label"].map(label2id)

    # 划分训练/测试集
    from sklearn.model_selection import train_test_split

    train_df, test_df = train_test_split(
        df, test_size=0.2, random_state=RANDOM_SEED, stratify=df["label_id"]
    )

    train_ds = Dataset.from_pandas(
        train_df[["text", "label_id"]].rename(columns={"label_id": "label"})
    )
    test_ds = Dataset.from_pandas(
        test_df[["text", "label_id"]].rename(columns={"label_id": "label"})
    )

    print(f"📦 训练集: {len(train_ds)}，测试集: {len(test_ds)}")
    return DatasetDict({"train": train_ds, "test": test_ds}), label2id, id2label


# =================================== 评估函数 ===================================


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {
        "accuracy": accuracy_score(labels, predictions),
        "f1": f1_score(labels, predictions, average="weighted"),
    }


# =================================== 开始训练 ===================================

print("🎯 开始 hfl/chinese-bert-wwm-ext 基线训练（Notebook 版）\n")

# 1. 准备数据
dataset, label2id, id2label = prepare_dataset()
num_labels = len(label2id)

# 2. Tokenizer
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)


def tokenize_function(examples):
    return tokenizer(
        examples["text"], truncation=True, padding="max_length", max_length=MAX_LENGTH
    )


tokenized_datasets = dataset.map(tokenize_function, batched=True)

# 3. 模型
model = AutoModelForSequenceClassification.from_pretrained(
    MODEL_NAME, num_labels=num_labels, label2id=label2id, id2label=id2label
)

# 4. 训练参数（兼容 Transformers 4.57+）
training_args = TrainingArguments(
    output_dir=str(OUTPUT_DIR),
    eval_strategy="epoch",  # 新版参数名
    save_strategy="epoch",
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=EPOCHS,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    report_to="none",
    logging_steps=50,
    seed=RANDOM_SEED,
)

# 5. Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    compute_metrics=compute_metrics,
)

# 6. 开始训练
print("\n🚀 开始训练...")
trainer.train()

# 7. 最终评估
print("\n📊 最终评估结果:")
results = trainer.evaluate()
print(f"   Accuracy: {results['eval_accuracy']:.4f}")
print(f"   F1 (weighted): {results['eval_f1']:.4f}")

# 8. 保存模型
print("\n💾 正在保存模型...")
trainer.save_model(str(OUTPUT_DIR))
tokenizer.save_pretrained(str(OUTPUT_DIR))

# 保存意图映射
with open(OUTPUT_DIR / "intent_mapping.json", "w", encoding="utf-8") as f:
    json.dump(
        {"intent2id": label2id, "id2intent": id2label}, f, ensure_ascii=False, indent=2
    )

with open(OUTPUT_DIR / "intents.txt", "w", encoding="utf-8") as f:
    f.write("微信聊天意图列表\n" + "=" * 50 + "\n\n")
    for i, intent in enumerate(sorted(id2label.values())):
        f.write(f"{i+1:2d}. {intent}\n")

print(f"\n✅ 基线训练完成！模型已保存到：")
print(f"   {OUTPUT_DIR}")

🎯 开始 hfl/chinese-bert-wwm-ext 基线训练（Notebook 版）


📊 开始准备数据集...
📂 正在加载: telemarketing_intent_cn.jsonl
📂 正在加载: crosswoz.jsonl
✅ 已合并 CrossWOZ，总样本: 20279
   黑名单过滤: 20279 → 19287
   样本阈值过滤后: 19149 样本，70 意图
   数据平衡后: 9782 样本
📦 训练集: 7825，测试集: 1957


Map: 100%|██████████| 1957/1957 [00:00<00:00, 30786.22 examples/s]
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at hfl/chinese-bert-wwm-ext and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



🚀 开始训练...


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,2.241000,1.598686,0.754727,0.723348
2,1.100900,0.946527,0.848748,0.839414
3,0.742100,0.749691,0.873786,0.868346
4,0.611700,0.697131,0.874808,0.870780



📊 最终评估结果:


   Accuracy: 0.8748
   F1 (weighted): 0.8708

💾 正在保存模型...

✅ 基线训练完成！模型已保存到：
   C:\Users\ASUS\Desktop\Yuzuriha-Rin\assets\models\chinese_bert_baseline
